In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [ ]:
# creating empty google jobs df 

google_jobs_df = pd.DataFrame()

In [ ]:
def scrape_google_jobs(google_jobs_df, postings):

    # using sing selenium to launch and scroll through the Google Jobs page
    # url filters: seacrh query "jobs" posted within past day, within 5 miles of New York, NY
    url = 'https://www.google.com/search?q=jobs&oq=google+jobs+data+analyst&aqs=chrome..69i57j69i59j0i512j0i22i30i625l4j69i60.4543j0j7&sourceid=chrome&ie=UTF-8&ibp=htl;jobs&sa=X&ved=2ahUKEwjXsv-_iZP9AhVPRmwGHX5xDEsQutcGKAF6BAgPEAU&sxsrf=AJOqlzWGHNISzgpAUCZBmQA1mWXXt3I7gA:1676311105893#fpstate=tldetail&htivrt=jobs&htichips=city:Owg_06VPwoli_nfhBo8LyA%3D%3D&htischips=city;Owg_06VPwoli_nfhBo8LyA%3D%3D:New%20York_comma_%20NY&htilrad=8.0467&htidocid=1R7H3j_x2GhpVLK0AAAAAA%3D%3D'
    driver = webdriver.Chrome()
    driver.get(url)

    xpaths = {
     'Role'            :"./div[2]",
     'Company'         :"./div[4]/div/div[1]",
     'Location'        :"./div[4]/div/div[2]",
     'Source'          :"./div[4]/div/div[3]",
     'Posted'          :".//*[name()='path'][contains(@d,'M11.99')]/ancestor::div[1]",
     'Full / Part Time':".//*[name()='path'][contains(@d,'M20 6')]/ancestor::div[1]",
     'Scraped Salary'  :".//span[@class='LL4CDc' and contains(@aria-label,'Salary')]/span",
     'Job Highlights'  :"./div/div[4]/div[1]/div[2]/g-expandable-container/div/g-expandable-content[2]/span"
    }

    jobs_to_do_scr = postings
    jobs_done_scr = 0
    data = {key:[] for key in xpaths}

    while jobs_done_scr < jobs_to_do_scr:
        lis_scr = driver.find_elements(By.XPATH, "//li[@data-ved]//div[@role='treeitem']/div/div")

        for li_scr in lis_scr[jobs_done_scr:jobs_done_scr+1]:
            driver.execute_script('arguments[0].scrollIntoView({block: "center", behavior: "smooth"});', li_scr)

            for key in xpaths:

                if key != 'Job Highlights':

                    try:
                        t = li_scr.find_element(By.XPATH, xpaths[key]).get_attribute('innerText')
                    except NoSuchElementException:
                        t = '*missing data*'
                    data[key].append(t)

            jobs_done_scr += 1
            print(f'{jobs_done_scr=}', end='\r')
            time.sleep(.2)   
    
    jobs_to_do_descr = postings
    jobs_done_descr = 0

    while jobs_done_descr < jobs_to_do_descr:
        lis_descr = driver.find_elements(By.XPATH, "//*[@id='gws-plugins-horizon-jobs__job_details_page']")

        for li_descr in lis_descr[jobs_done_descr:jobs_done_descr+1]:
            driver.execute_script('arguments[0].scrollIntoView({block: "center", behavior: "smooth"});', li_descr)

            for key in xpaths:

                if key == 'Job Highlights':

                    try:
                        t = li_descr.find_element(By.XPATH, xpaths[key]).get_attribute('innerText')
                    except NoSuchElementException:
                        t = '*missing data*'
                    data[key].append(t)

            jobs_done_descr += 1
            print(f'{jobs_done_descr=}', end='\r')
            time.sleep(.2)

    scraped_df = pd.DataFrame(data)
    google_jobs_df = google_jobs_df.append(scraped_df)
    # google_jobs_df = google_jobs_df.drop_duplicates()
    google_jobs_df = google_jobs_df.reset_index().drop(columns=['index'])

    return google_jobs_df

# if want to iterate

# loops_to_do = 3
# loops_done = 0

# while loops_done < loops_to_do:
    
#     google_jobs_df = scrape_google_jobs(google_jobs_df, 150)
    
#     loops_done += 1
    
# google_jobs_df

In [ ]:
google_jobs_df = scrape_google_jobs(google_jobs_df, 10)

In [ ]:
def find_salaries(df):
    
    df['Salary From Description'] = '' # creating column where salaries found in description will be placed
    df['Salary Lower Bound'] = '' # lower bound
    df['Salary Upper Bound'] = '' # upper bound
    
    for ind in df.index: # iterating through the descriptions for each job

        salary_list = []

        phrase = df['Job Highlights'][ind] # text in description
        phrase = phrase.replace('•',' ') # eliminating bullet points
        phrase_to_list = phrase.split(" ") # creating list of all strings in the description

        for i in range(len(phrase_to_list)): # iterating through phrase_to_list

            if len(phrase_to_list[i]) > 0: # otherwise -1 index will break code
                
                if phrase_to_list[i] not in ['401K', '401k']: # eliminating '401K' so doesn't get labeled as salary
                    
                    if not [True if e in phrase_to_list[i].lower() else False for e in ['b','m']][0]: # weeding out cases like '$10b industry' or '$3M company'
                    
                        if ('$' in phrase_to_list[i]) or ('k' == phrase_to_list[i][-1].lower() and phrase_to_list[i][-2].isdigit()): # looking for salary indicators

                            salary_list.append(phrase_to_list[i]) # adding strings with salary indicators to ongoing list

                        if phrase_to_list[i] == '$' and phrase_to_list[i+1][0].isdigit(): # if there's a space between $ and number

                            salary_list.append(phrase_to_list[i+1]) # add the subsequent list item in this case

                        df['Salary From Description'][ind] = salary_list # place in this new column
          
        # now that salaries are collected from the description, creating lower and upper bounds based on given info
                
        if df['Scraped Salary'][ind] == '*missing data*': # if salary not initially scraped
            
            if df['Salary From Description'][ind] == []: # if no salary found in description
        
                df['Salary Lower Bound'][ind] = '*no salary found*'
                df['Salary Upper Bound'][ind] = '*no salary found*'
            
            else: # if salary is found in the description
                
                # if first value is just $, remove so doesn't affect making the range (cases where there's originally a space between $ and number)
                for i in df['Salary From Description'][ind]:
                    if i == '$':
                        df['Salary From Description'][ind].remove('$') 
                
                if len(df['Salary From Description'][ind]) > 2: # when multiple numbers are found, hard to tell what salary range is
                    
                    df['Salary Lower Bound'][ind] = '*salary ambiguous*'
                    df['Salary Upper Bound'][ind] = '*salary ambiguous*'
                
                if len(df['Salary From Description'][ind]) == 2: # if two entries provided, create salary range 
                
                     # in cases where there are two "numbers" that are really 1-2 ranges -> [20K-30K,5K-6K] or [20K,15K-16K]
        
                    if ('–' in df['Salary From Description'][ind][0]) or ('-' in df['Salary From Description'][ind][0]) or ('–' in df['Salary From Description'][ind][1]) or ('-' in df['Salary From Description'][ind][1]) or ('—' in df['Salary From Description'][ind][0]) or ('—' in df['Salary From Description'][ind][1]): 

                        df['Salary Lower Bound'][ind] = '*salary ambiguous*' # ambiguous cause there are actually more than 2 numbers in this list
                        df['Salary Upper Bound'][ind] = '*salary ambiguous*'

                    else: # otherwise, clear to create salary range

                        df['Salary Lower Bound'][ind] = df['Salary From Description'][ind][0] 
                        df['Salary Upper Bound'][ind] = df['Salary From Description'][ind][1]
                
                elif len(df['Salary From Description'][ind]) == 1: # one entry
                    
                    # checking for cases where there is only "one" entry but truly two numbers -> ['10K-20K']
                    # if truly just one number, just include that number -> ['$60,000']
                    
                    if ('–' not in df['Salary From Description'][ind][0]) and ('-' not in df['Salary From Description'][ind][0]) and ('—' not in df['Salary From Description'][ind][0]): 
                        
                        df['Salary Lower Bound'][ind] = df['Salary From Description'][ind][0] 
                     
                    else: # otherwise, have to isolate the lower and upper bound
                    
                        if '–' in df['Salary From Description'][ind][0]: 
                            desc_sal = df['Salary From Description'][ind][0].replace('–', ' ')
                        elif '—' in df['Salary From Description'][ind][0]:
                            desc_sal = df['Salary From Description'][ind][0].replace('—', ' ')
                        elif '-' in df['Salary From Description'][ind][0]:
                            desc_sal = df['Salary From Description'][ind][0].replace('-', ' ')

                        desc_sal_list = desc_sal.split(' ')

                        df['Salary Lower Bound'][ind] = desc_sal_list[0]
                        df['Salary Upper Bound'][ind] = desc_sal_list[1]
            
        else: # if salary found in initial scrape, use it as final salary 
            
            if '–' in df['Scraped Salary'][ind]: # removing dashes
                scraped_sal = df['Scraped Salary'][ind].replace('–', ' ')
            elif '—' in df['Scraped Salary'][ind]: # removing dashes
                scraped_sal = df['Scraped Salary'][ind].replace('—', ' ')
            elif '-' in df['Scraped Salary'][ind]:
                scraped_sal = df['Scraped Salary'][ind].replace('-', ' ')
            else:
                scraped_sal = df['Scraped Salary'][ind]
                
            scraped_sal_list = scraped_sal.split(' ') # seperating into list of strings
            
            df['Salary Lower Bound'][ind] = scraped_sal_list[0] # first number
            df['Salary Upper Bound'][ind] = scraped_sal_list[1] # second number

    return df

In [ ]:
google_jobs_df_add_sal = find_salaries(google_jobs_df)

In [ ]:
def clean_salaries(df):
    
    df['Salary Magnitude'] = '' 
    df['Salary Range'] = ''
    df['Salary Ratio'] = ''
    
    for ind in df.index:
        
        lower = df['Salary Lower Bound'][ind]
        upper = df['Salary Upper Bound'][ind]
        
        if df['Salary Lower Bound'][ind] not in ['*no salary found*','*salary ambiguous*']: # only for entries that have values
            
            df['Salary Magnitude'][ind] = [1000 if (('k' in lower.lower()) or ('k' in upper.lower())) else 1][0] # if 'K' in entry, will mutliply by 1000            
            
            # removing all non-numerical characters unless decimal is present, in which case manually remove characters in the mapping_table so decimal is not deleted

            df['Salary Lower Bound'][ind] = re.sub(r'[^0-9.]', '', lower)
            df['Salary Upper Bound'][ind] = re.sub(r'[^0-9.]', '', upper)
    
            # if this cleaning leaves the entry emtpy, leave this message:
            
            df['Salary Lower Bound'][ind] =  ['*no number present*' if df['Salary Lower Bound'][ind] == '' else df['Salary Lower Bound'][ind]][0] 
            df['Salary Upper Bound'][ind] =  ['*no number present*' if df['Salary Upper Bound'][ind] == '' else df['Salary Upper Bound'][ind]][0] 
            
            # multiplying by 1 or 1000 depending on whether or not 'K' was present in original entry
            
            if df['Salary Lower Bound'][ind] != '*no number present*':
                
                df['Salary Lower Bound'][ind] = (float(df['Salary Lower Bound'][ind]) * df['Salary Magnitude'][ind])
            
            if df['Salary Upper Bound'][ind] != '*no number present*':
    
                df['Salary Upper Bound'][ind] = float(df['Salary Upper Bound'][ind]) * df['Salary Magnitude'][ind]
            
            # if there are two numbers, create column with polished range, add ratio to new col
            
            if (df['Salary Lower Bound'][ind] != '*no number present*') and (df['Salary Upper Bound'][ind] != '*no number present*'):
                        
                # in cases where they were placed in wrong order

                if df['Salary Lower Bound'][ind] > df['Salary Upper Bound'][ind]: 
            
                    real_upper = df['Salary Lower Bound'][ind] 
                    real_lower = df['Salary Upper Bound'][ind]

                    df['Salary Lower Bound'][ind] = real_lower # switching which column each is placed in
                    df['Salary Upper Bound'][ind] = real_upper
                
                df['Salary Range'][ind] = '${:,.2f}'.format(df['Salary Lower Bound'][ind]) + '-' + '${:,.2f}'.format(df['Salary Upper Bound'][ind])
                df['Salary Ratio'][ind] = round((df['Salary Upper Bound'][ind] / df['Salary Lower Bound'][ind]),1)
                
            # if just one value, include that value alone, with ratio set to 1
            
            elif (df['Salary Lower Bound'][ind] != '*no number present*') and (df['Salary Upper Bound'][ind] == '*no number present*'):
                
                df['Salary Range'][ind] = '${:,.2f}'.format(df['Salary Lower Bound'][ind])
                df['Salary Ratio'][ind] = 1.0
            
        if df['Salary Lower Bound'][ind] == '*no salary found*': # if no salary found
            
            df['Salary Range'][ind] = '*no salary found*'
            df['Salary Ratio'][ind] = '*no salary found*'
                
        if df['Salary Lower Bound'][ind] == '*salary ambiguous*': # if no salary found

            df['Salary Range'][ind] = '*salary ambiguous*'
            df['Salary Ratio'][ind] = '*salary ambiguous*'
                
    df = df.drop(columns=['Salary Magnitude'])
                   
    return df
    

In [ ]:
google_jobs_df_cleaned = clean_salaries(google_jobs_df_add_sal)

In [ ]:
google_jobs_df_cleaned.loc[:59]

In [ ]:
print('Entries with Salaries Found:', len(google_jobs_df_cleaned[(google_jobs_df_cleaned['Salary Range'] != '*no salary found*')]))
print('Total Entries:', len(google_jobs_df_cleaned))

In [ ]:
google_jobs_df_cleaned['Location'].unique()

In [ ]:
google_jobs_df_cleaned['Source'].unique()

In [ ]:
# entries without salaries come from these sources

google_jobs_df_cleaned[(google_jobs_df_cleaned['Salary Range'] == '*no salary found*')].value_counts('Source')

In [ ]:
x = google_jobs_df_cleaned[(google_jobs_df_cleaned['Salary Ratio'] != '*no salary found*') & (google_jobs_df_cleaned['Salary Range'] != '*salary ambiguous*')]['Salary Ratio'].to_list()

plt.hist(x)